In [1]:
#%matplotlib inline
import tensorflow as tf
import numpy as np
#import matplotlib.pyplot as plt
from siamese.dataset import Dataset
from siamese.dataset import DataLoader
from siamese.model import *
from extensies import metrics as mt
from extensies import preprocessing as ps
from matplotlib import gridspec

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import classification_report
import pandas as pd 
import ast
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

columns = ['date','name','modality','size','model','num_of_data','iterations','augmentation',
           'normalization','acc','acc_w','acc_p','auc','auc_w','auc_p','komentar']
pd.set_option('display.max_rows', 500)
df = pd.read_csv('records.csv',index_col=0)


In [2]:
def print_scores(labels, predicted_labels):
    print("Accuracy: {}".format(accuracy_score(labels, predicted_labels)))
    print("Precision: {}".format(precision_score(labels, predicted_labels)))
    print("Recall: {}".format(recall_score(labels, predicted_labels)))
    print("F1: {}".format(f1_score(labels, predicted_labels)))
    
def print_all_models(df):

    kfold_indecies = df[df['kfold_id'].isna() == False]['kfold_id'].unique()
    for idx in kfold_indecies:
        means = df[df['kfold_id'] == idx].mean()
        name = df[df['kfold_id'] == idx].iloc[0]['name']
        iters = df[df['kfold_id'] == idx].iloc[0]['iterations']
        print("Model name: ",name,", Iter: ",iters,", ACC: {0:.4f}".format(means['acc_p']),", AUC: {0:.4f}".format(means['auc_p']), ",   index:",idx)

from collections import Counter
def undersample(images,labels):
    num_of_pos = Counter(labels)[1]
    indecies0 = np.where(labels == 0)
    indecies1 = np.where(labels == 1)[0]
    indecies0 = np.random.choice(indecies0[0], num_of_pos)
    indecies = np.concatenate((indecies0, indecies1))
    
    return images[indecies],labels[indecies]

## Listing of all trained models

In [5]:
df.tail(40)

,acc,acc_p,acc_w,auc,auc_p,auc_w,augmentation,date,iterations,kfold_id,komentar,modality,model,name,normalization,num_of_data,size
222,0.830508,0.779661,0.169492,0.437500,0.568452,0.562500,False,2019-05-02,700,16745.0,NaN,['adc/t/40x40x1'],siamese,adc_40x40x1_90015,ScaleNormalization,296,40x40x1
223,0.610169,0.610169,0.474576,0.531401,0.547101,0.530193,False,2019-05-02,700,16745.0,NaN,['adc/t/40x40x1'],siamese,adc_40x40x1_56621,ScaleNormalization,296,40x40x1
224,0.783333,0.750000,0.466667,0.633333,0.633333,0.555556,False,2019-05-02,700,77044.0,NaN,['adc/t/40x40x1'],xmas,adc_40x40x1_52590,ScaleNormalization,296,40x40x1
225,0.847458,0.779661,0.254237,0.682222,0.687778,0.514444,False,2019-05-02,700,77044.0,NaN,['adc/t/40x40x1'],xmas,adc_40x40x1_49374,ScaleNormalization,296,40x40x1
226,0.661017,0.661017,0.542373,0.534461,0.566416,0.612782,False,2019-05-02,700,77044.0,NaN,['adc/t/40x40x1'],xmas,adc_40x40x1_63533,ScaleNormalization,296,40x40x1
227,0.830508,0.677966,0.084746,0.437500,0.672619,0.360119,False,2019-05-02,700,77044.0,NaN,['adc/t/40x40x1'],xmas,adc_40x40x1_44954,ScaleNormalization,296,40x40x1
228,0.694915,0.644068,0.525424,0.632246,0.590580,0.564010,False,2019-05-02,700,77044.0,NaN,['adc/t/40x40x1'],xmas,adc_40x40x1_62415,ScaleNormalization,296,40x40x1
229,0.783333,0.750000,0.366667,0.588889,0.633333,0.488889,False,2019-05-02,700,28096.0,NaN,['t2tsetra/t/40x40x1'],xmas,t2tsetra_40x40x1_88765,ScaleNormalization,296,40x40x1
230,0.864407,0.830508,0.271186,0.555556,0.672222,0.570000,False,2019-05-02,700,28096.0,NaN,['t2tsetra/t/40x40x1'],xmas,t2tsetra_40x40x1_34598,ScaleNormalization,296,40x40x1
231,0.644068,0.694915,0.474576,0.500000,0.592732,0.570802,False,2019-05-02,700,28096.0,NaN,['t2tsetra/t/40x40x1'],xmas,t2tsetra_40x40x1_82227,ScaleNormalization,296,40x40x1


## Shorter list of average of kfold valided models

In [4]:
print_all_models(df)

Model name:  adc_t2tsetra_20x20x1_44816 , Iter:  200 , ACC: 0.6384 , AUC: 0.4639 ,   index: 39849.0
Model name:  adc_t2tsetra_20x20x1_23981 , Iter:  200 , ACC: 0.6858 , AUC: 0.5030 ,   index: 87907.0
Model name:  adc_t2tsetra_20x20x1_67378 , Iter:  200 , ACC: nan , AUC: nan ,   index: 19399.0
Model name:  adc_t2tsetra_20x20x1_86877 , Iter:  200 , ACC: nan , AUC: nan ,   index: 17244.0
Model name:  adc_t2tsetra_20x20x1_89062 , Iter:  200 , ACC: nan , AUC: nan ,   index: 59765.0
Model name:  adc_t2tsetra_20x20x1_86718 , Iter:  200 , ACC: 0.7059 , AUC: 0.5231 ,   index: 52709.0
Model name:  adc_t2tsetra_ktrans_40x40x1_24314 , Iter:  1500 , ACC: 0.7214 , AUC: 0.5874 ,   index: 37356.0
Model name:  adc_t2tsetra_ktrans_40x40x1_13183 , Iter:  1500 , ACC: 0.7083 , AUC: 0.5837 ,   index: 51107.0
Model name:  t2tsetra_40x40x1_44045 , Iter:  1000 , ACC: 0.6903 , AUC: 0.5468 ,   index: 21295.0
Model name:  t2tsetra_40x40x1_17248 , Iter:  1000 , ACC: 0.6527 , AUC: 0.4964 ,   index: 57174.0
Model na

## Kfold evaluation

### Load data and get feature maps for each model
To the variable <strong>idxs</strong> add indecies of all models from upside listing, index: 83926

In [6]:

datasets = {}
idxs = [61266,80407,18509]
for idx in idxs:
    kf_idxs = df[df['kfold_id']== idx].index
    modalities = df.iloc[kf_idxs[0]]['modality']
    modalities = ast.literal_eval(modalities)
    
    if len(modalities) > 1:
        loader = DataLoader('../../data/', modalities)
        loader.load_data()
        loader.combine_channels(modalities)
        kfold = loader.k_fold(subdir = 'combined',num = 5)
        data_shape = loader.get_shape('combined')
        
    else:
        loader = DataLoader('../../data/', modalities)
        loader.load_data()
        kfold = loader.k_fold(subdir = modalities[0],num = 5)
        data_shape = loader.get_shape(modalities[0])
    
    for idx in kf_idxs:
        
        model_name = df.iloc[idx]['name']       
        print('Model name: ',model_name)
        X_train,X_test,y_train,y_test = next(kfold)
        
#         X_train,y_train = undersample(X_train,y_train)
        
        datasets[idx] = Dataset()
        datasets[idx].images_train = X_train
        datasets[idx].images_test = X_test
        datasets[idx].labels_train = y_train
        datasets[idx].labels_test = y_test
        
img_placeholder = tf.placeholder(tf.float32, [None] + list(data_shape[1:]), name='img')
net = xmas_model(img_placeholder, reuse=False)        

train_feat = {}
search_feat = {}
for idx in idxs:
    kf_idxs = df[df['kfold_id']== idx].index    
    for idx in kf_idxs:
        saver = tf.train.Saver()
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            ckpt = tf.train.get_checkpoint_state("model")
            saver.restore(sess, "model/" + df.iloc[idx]['name'] + ".ckpt")
            train_feat[idx] = sess.run(net, feed_dict={img_placeholder:datasets[idx].images_train})
            search_feat[idx] = sess.run(net, feed_dict={img_placeholder:datasets[idx].images_test})

Model name:  adc_cbval_t2tsetra_20x20x1_58623
Model name:  adc_cbval_t2tsetra_20x20x1_10291
Model name:  adc_cbval_t2tsetra_20x20x1_62249
Model name:  adc_cbval_t2tsetra_20x20x1_69277
Model name:  adc_cbval_t2tsetra_20x20x1_78051
Model name:  adc_cbval_t2tsetra_20x20x1_85988
Model name:  adc_cbval_t2tsetra_20x20x1_73399
Model name:  adc_cbval_t2tsetra_20x20x1_30498
Model name:  adc_cbval_t2tsetra_20x20x1_91262
Model name:  adc_cbval_t2tsetra_20x20x1_35266
Model name:  adc_cbval_t2tsetra_20x20x1_86124
Model name:  adc_cbval_t2tsetra_20x20x1_69371
Model name:  adc_cbval_t2tsetra_20x20x1_46919
Model name:  adc_cbval_t2tsetra_20x20x1_97964
Model name:  adc_cbval_t2tsetra_20x20x1_83582
INFO:tensorflow:Restoring parameters from model/adc_cbval_t2tsetra_20x20x1_58623.ckpt
INFO:tensorflow:Restoring parameters from model/adc_cbval_t2tsetra_20x20x1_10291.ckpt
INFO:tensorflow:Restoring parameters from model/adc_cbval_t2tsetra_20x20x1_62249.ckpt
INFO:tensorflow:Restoring parameters from model/adc_

### Use metrics to get predictions
we use our metrics to get labels for test data by measuring euclidean distance

In [7]:
y_pred = {}
y_pred_t = {}
y_pred_p = {}
for x in idxs:
    kf_idxs = df[df['kfold_id']== x].index
    for idx in kf_idxs:
        y_pred[idx] = []
        y_pred_t[idx] = []
        y_pred_p[idx] = []
    for idx in kf_idxs:
        for feat in search_feat[idx]:
            #calculate the cosine similarity and sort
            y_pred[idx].append(mt.siamese_predict(train_feat[idx],feat,datasets[idx]))
            y_pred_t[idx].append(mt.treshold_predict(train_feat[idx],feat,datasets[idx],0.4,10,distance= 'euclidean'))
            y_pred_p[idx].append(mt.percent_predict(train_feat[idx],feat,datasets[idx],10,distance= 'euclidean'))
        y_pred[idx] = np.array(y_pred[idx])
        y_pred_t[idx] = np.array(y_pred_t[idx])
        y_pred_p[idx] = np.array(y_pred_p[idx])

# Show accuracy for each model

In [10]:
for x in idxs:
    kf_idxs = df[df['kfold_id']== x].index
    acc = 0
    auc2 = 0
    for x in kf_idxs:
        acc += accuracy_score(datasets[x].labels_test,y_pred_t[x])
        auc2 += metrics.roc_auc_score(datasets[x].labels_test,y_pred_p[x])
    
    print(' avg accuracy: ',acc / len(kf_idxs),' auc:',auc2 / len(kf_idxs))

 avg accuracy:  0.7196045197740113  auc: 0.6481494557650164
 avg accuracy:  0.7363841807909604  auc: 0.6969989708570944
 avg accuracy:  0.7532203389830509  auc: 0.6925866417250857


### Prediction with 3 models

In [11]:
zips = zip(df[df['kfold_id']== 61266].index, df[df['kfold_id']== 80407].index, df[df['kfold_id']== 18509].index)
auc = 0
acc = 0
for x,y,z in zips:
    prob = [(i+j+k)/3 for i,j,k in zip(y_pred_p[x],y_pred_p[y],y_pred_p[z])]
    prob2 = [1 if x >=0.5 else 0 for x in prob]
    auc += metrics.roc_auc_score(datasets[x].labels_test,prob)
    acc += accuracy_score(datasets[x].labels_test,prob2)

print("AUC: ", auc / 5)
print("ACC: ", acc / 5)

AUC:  0.7190883791604614
ACC:  0.7635028248587571
